In [1]:
# Cell 1: imports & paths
import os, re, time, json, mimetypes
from pathlib import Path
from typing import Optional, Tuple, Dict, Any, List

import pandas as pd
import requests

ROOT = Path(".")
CSV_IN = ROOT / "map_data_demo.csv"          # change if needed
PUBLIC = ROOT / "public"
LOGO_DIR = PUBLIC / "logos"
DATA_DIR = PUBLIC / "data"

LOGO_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

PLACEHOLDER_WEB = "/logos/placeholder.png"   # put a placeholder file here
BRANDFETCH_KEY = ""  # <= add your key if you have one, else leave empty

# basic knobs
RATE_DELAY_SEC = 0.25     # be polite to free endpoints
TIMEOUT = 6               # seconds for HTTP calls


In [2]:
# Cell 2: helpers

def norm_domain(url: str) -> str:
    if not isinstance(url, str) or not url.strip():
        return ""
    u = url.strip().lower()
    u = re.sub(r"^https?://", "", u)
    u = re.sub(r"^www\.", "", u)
    u = u.split("/")[0]
    return u

def slugify(name: str) -> str:
    return re.sub(r"[^a-z0-9]+", "-", (name or "logo").lower()).strip("-")[:60]

def pick_ext_from_headers(content_type: str, url: str) -> str:
    if content_type:
        ct = content_type.split(";")[0].strip()
        ext = mimetypes.guess_extension(ct) or ""
        if ext in (".svg", ".png", ".jpg", ".jpeg", ".webp", ".ico"):
            return ext
    for ext in (".svg", ".png", ".webp", ".jpg", ".jpeg", ".ico"):
        if url.lower().endswith(ext):
            return ext
    return ".png"

def fetch_binary(url: str, headers: Optional[Dict[str, str]] = None, timeout: int = TIMEOUT):
    try:
        r = requests.get(url, headers=headers or {}, timeout=timeout)
        if r.status_code == 200 and r.content and "text/html" not in r.headers.get("content-type", ""):
            return r
    except Exception:
        return None
    return None

def save_logo(resp, src_url: str, filename_base: str) -> str:
    ext = pick_ext_from_headers(resp.headers.get("content-type", ""), src_url)
    path = LOGO_DIR / f"{filename_base}{ext}"
    with open(path, "wb") as f:
        f.write(resp.content)
    return f"/logos/{path.name}"  # web path served by Vercel


In [3]:
# Cell 3: brandfetch selection & source fetchers

def select_best_brandfetch_logo(payload: Dict[str, Any]) -> Optional[str]:
    """
    Pick the best logo 'src' from Brandfetch response using type/format/size priority.
    """
    logos = payload.get("logos") or []
    if not logos:
        return None

    type_rank = {"full": 3, "wordmark": 2, "symbol": 1, "icon": 1}
    fmt_rank = {"svg": 5, "png": 4, "webp": 3, "jpg": 2, "jpeg": 2, "ico": 1}

    candidates = []
    for lg in logos:
        ltype = lg.get("type", "").lower()
        for fmt in lg.get("formats", []):
            src = fmt.get("src")
            if not src:
                continue
            fmt_ext = fmt.get("format", "").lower() or src.split("?")[0].split(".")[-1].lower()
            w = fmt.get("width") or 0
            h = fmt.get("height") or 0
            candidates.append({
                "src": src,
                "type_score": type_rank.get(ltype, 0),
                "fmt_score": fmt_rank.get(fmt_ext, 0),
                "area": (w or 0) * (h or 0)
            })

    if not candidates:
        return None

    candidates.sort(key=lambda c: (c["type_score"], c["fmt_score"], c["area"]), reverse=True)
    return candidates[0]["src"]

def try_brandfetch(domain: str) -> Optional[str]:
    if not BRANDFETCH_KEY:
        return None
    url = f"https://api.brandfetch.io/v2/brands/{domain}"
    try:
        r = requests.get(url, headers={"Authorization": f"Bearer {BRANDFETCH_KEY}"}, timeout=TIMEOUT)
        if r.status_code == 200:
            src = select_best_brandfetch_logo(r.json())
            return src
    except Exception:
        pass
    return None

def try_clearbit(domain: str) -> Optional[str]:
    url = f"https://logo.clearbit.com/{domain}"
    r = fetch_binary(url, timeout=TIMEOUT)
    return url if r else None

def try_duckduckgo(domain: str) -> Optional[str]:
    url = f"https://icons.duckduckgo.com/ip3/{domain}.ico"
    r = fetch_binary(url, timeout=TIMEOUT)
    return url if r else None


In [4]:
# Cell 4: resolver with preference order

def resolve_logo_url(domain: str) -> Tuple[str, str]:
    """
    Return (url, source_name).
    Preference:
      - Brandfetch (if key present) > Clearbit > DuckDuckGo
    """
    if not domain:
        return "", "none"

    # Try Brandfetch first if you provided a key (best quality)
    if BRANDFETCH_KEY:
        bf = try_brandfetch(domain)
        if bf: return bf, "Brandfetch"

    # Then Clearbit (fast, good coverage)
    cb = try_clearbit(domain)
    if cb: return cb, "Clearbit"

    # Then DuckDuckGo favicon
    ddg = try_duckduckgo(domain)
    if ddg: return ddg, "DuckDuckGo"

    return "", "none"


In [5]:
# Cell 5: enrich CSV, download logos, build JSON for app

# --- load your data ---
df = pd.read_csv(CSV_IN)

# Column names (adjust if yours differ)
name_col   = "Entity"
domain_col = "Domain"
bucket_col = "Map Bucket 1 (Normalized)" if "Map Bucket 1 (Normalized)" in df.columns else "Map Bucket 1"
desc_col   = "Description" if "Description" in df.columns else None
relevance_col = "Relevance Flag (Y/N)"

# filter to relevance == Y
if relevance_col in df.columns:
    df = df[df[relevance_col].astype(str).str.upper().eq("Y")].copy()

df["__domain"] = df[domain_col].astype(str).map(norm_domain)
df["Logo_URL"] = ""
df["Logo_File"] = ""
df["Logo_Source"] = ""

cache: Dict[str, str] = {}   # domain -> saved /logos/.. path

for i, row in df.iterrows():
    domain = row["__domain"]
    name   = str(row.get(name_col, "")).strip()
    if not domain:
        df.at[i, "Logo_URL"] = ""
        df.at[i, "Logo_File"] = PLACEHOLDER_WEB
        df.at[i, "Logo_Source"] = "placeholder"
        continue

    # if already downloaded for same domain, reuse
    if domain in cache:
        df.at[i, "Logo_URL"] = ""
        df.at[i, "Logo_File"] = cache[domain]
        df.at[i, "Logo_Source"] = "cached"
        continue

    url, source = resolve_logo_url(domain)
    if url:
        # download binary so site works offline & on Vercel
        resp = fetch_binary(url, timeout=TIMEOUT)
        if resp:
            web_path = save_logo(resp, url, slugify(domain or name))
            df.at[i, "Logo_URL"] = url
            df.at[i, "Logo_File"] = web_path
            df.at[i, "Logo_Source"] = source
            cache[domain] = web_path
        else:
            df.at[i, "Logo_URL"] = ""
            df.at[i, "Logo_File"] = PLACEHOLDER_WEB
            df.at[i, "Logo_Source"] = "placeholder"
    else:
        df.at[i, "Logo_URL"] = ""
        df.at[i, "Logo_File"] = PLACEHOLDER_WEB
        df.at[i, "Logo_Source"] = "placeholder"

    time.sleep(RATE_DELAY_SEC)

# save enriched CSV
out_csv = ROOT / "map_data_with_logos.csv"
df.drop(columns=["__domain"], errors="ignore").to_csv(out_csv, index=False)
print(f"✅ Saved enriched CSV -> {out_csv}")

# build grouped JSON for the app: public/data/map_data.json
grouped: Dict[str, List[Dict[str, Any]]] = {}
for _, r in df.iterrows():
    bucket = str(r.get(bucket_col, "")).strip() or "Uncategorized"
    item = {
        "name": str(r.get(name_col, "")).strip(),
        "description": str(r.get(desc_col, "")).strip() if desc_col else "",
        "website": str(r.get(domain_col, "")).strip(),
        "logo": r.get("Logo_File") or PLACEHOLDER_WEB
    }
    grouped.setdefault(bucket, []).append(item)

out_json = DATA_DIR / "map_data.json"
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(grouped, f, ensure_ascii=False, indent=2)
print(f"✅ Wrote JSON for app -> {out_json}")
print(f"📁 Logos saved in -> {LOGO_DIR}")


✅ Saved enriched CSV -> map_data_with_logos.csv
✅ Wrote JSON for app -> public/data/map_data.json
📁 Logos saved in -> public/logos
